In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.stats as stats
import os
import joypy
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import gridspec
from scipy import stats

In [2]:
df=pd.read_table('Fig4B.csv',sep='\t')
df.head()

,SeqID,Sample,Genomecoverage,Windowcoverage,NRAMP1copy,Haplotype
0,21228,F10-1-3,42.444019,85.159800,2.006403,AAGACATAA-TD
1,21228,F10-1-3,42.444019,73.339576,1.727913,AAGACATAA-TD
2,21228,F10-1-3,42.444019,67.280899,1.585168,AAGACATAA-TD
3,21228,F10-1-3,42.444019,70.963795,1.671939,AAGACATAA-TD
4,21228,F10-1-3,42.444019,77.144819,1.817566,AAGACATAA-TD


In [3]:
df['Color']=np.select([df['Haplotype'] == 'AAGACATAA-TD',
                       df['Haplotype'] == 'ATG-TD', 
                       df['Haplotype'] == 'CT-TD', 
                       df['Haplotype'] == 'ancestral'],
                       ['#F1BB7B','#5B1A18','#FD6467','#3573B9'])
df.head()

,SeqID,Sample,Genomecoverage,Windowcoverage,NRAMP1copy,Haplotype,Color
0,21228,F10-1-3,42.444019,85.159800,2.006403,AAGACATAA-TD,#F1BB7B
1,21228,F10-1-3,42.444019,73.339576,1.727913,AAGACATAA-TD,#F1BB7B
2,21228,F10-1-3,42.444019,67.280899,1.585168,AAGACATAA-TD,#F1BB7B
3,21228,F10-1-3,42.444019,70.963795,1.671939,AAGACATAA-TD,#F1BB7B
4,21228,F10-1-3,42.444019,77.144819,1.817566,AAGACATAA-TD,#F1BB7B


In [4]:
df_grouped=df.groupby('Sample').agg(SeqID=('SeqID','first'),
                                       TDcopy=('NRAMP1copy','median'),
                                       Haplotype=('Haplotype','first'),
                                       STD=('NRAMP1copy','std')).reset_index()
df_grouped.head()

,Sample,SeqID,TDcopy,Haplotype,STD
0,F1-201,3968_AS,3.473466,ATG-TD,0.392767
1,F1-203,3968_AT,3.354630,ATG-TD,0.419511
2,F1-3,13184,2.696935,ATG-TD,0.731302
3,F1-7,3541_AE,3.123581,ATG-TD,0.327229
4,F10-1-3,21228,1.857687,AAGACATAA-TD,0.180153


In [5]:
df_grouped['HapSorted']=np.select([df_grouped['Haplotype'] == 'AAGACATAA-TD',
                       df_grouped['Haplotype'] == 'ATG-TD', 
                       df_grouped['Haplotype'] == 'CT-TD', 
                       df_grouped['Haplotype'] == 'Ancestral'],
                       [0,1,2,3])
df_grouped.sort_values('HapSorted')

,Sample,SeqID,TDcopy,Haplotype,STD,HapSorted
69,F15-4,3541_C,3.104428,AAGACATAA-TD,0.326574,0
25,F11-10,22610,4.188087,AAGACATAA-TD,0.436655,0
27,F11-6,3619_L,4.544243,AAGACATAA-TD,0.511423,0
28,F12-102,3619_H,3.304764,AAGACATAA-TD,0.425253,0
59,F13-T1-1115-12,3968_BU,2.584607,AAGACATAA-TD,0.351019,0
...,...,...,...,...,...,...
110,F4-2,3541_P,1.870655,CT-TD,0.180181,2
17,F10-13-4,3968_BE,1.043452,Ancestral,0.230264,3
16,F10-13-3,3968_BD,1.016585,Ancestral,0.216855,3
15,F10-13-2,3536_F,0.989814,Ancestral,0.158679,3


In [6]:
orderAccession=df_grouped.sort_values(['HapSorted','TDcopy']).agg('Sample').to_list()

In [7]:
for idx, row in df.iterrows():
    for index, value in enumerate(orderAccession):
        if row['Sample']==value:
            df.loc[idx,'Index'] = index

In [8]:
cmap = plt.cm.Greys
cmaplist = [cmap(i) for i in range(cmap.N) if i < 146]
for i in range(45):
    cmaplist[i] = (0.9450980392156862, 0.7333333333333333, 0.4823529411764706, 1.0)
for i in range(96):
    cmaplist[i+45] = (0.3568627450980392, 0.10196078431372549, 0.09411764705882353, 1.0)
cmaplist[141] = (0.9921568627450981, 0.39215686274509803, 0.403921568627451, 1.0)
for i in range(4):
    cmaplist[i+142] = (0.20784313725490197, 0.45098039215686275, 0.7254901960784313, 1.0)
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, len(cmaplist))

In [9]:
cm = 1/2.54
fig , ax = joypy.joyplot(df, by='Index', 
                         column='NRAMP1copy', 
                         colormap=cmap,
                         ylabels=False,
                         linewidth=.2,
                         xlabelsize=8,
                         figsize=(9*cm,12*cm))
plt.savefig("Fig4B.png", format="png",bbox_inches="tight",facecolor='w',dpi=800)